In [ ]:
# hide

%load_ext nb_black
%config InlineBackend.figure_format="retina"

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate import servers
from will_it_saturate import clients

# from will_it_saturate.gevent_client import GeventClient
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables

<IPython.core.display.Javascript object>

In [ ]:
control_host = "staging.wersdoerfer.de"
control_port = 8100

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

# control_host = "localhost"
control_host = "staging.wersdoerfer.de"
control_port = 8100

client_control_host = Host(name=control_host, port=control_port)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
benchmark_clients = [
    clients.WrkClient(name="wrk"),
    # clients.AioHttpClient(name="aiohttp"),
    clients.HttpxClient(name="httpx"),
]
# benchmark_client = clients.WrkClient(name="wrk")

server_control_host = Host(name=control_host, port=control_port)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
server_classes = [
    servers.NginxFileServer,
    servers.FastAPIUvicornServer,
    servers.NginxSendfileServer,
    servers.CaddyFileServer,
]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

byte, file_size, duration = 8, 10**5, 3
for server_class in server_classes:
    server = server_control_client.get_or_create_server(
        server_class(host="localhost", port=5001)
    )
    epoch = Epoch(file_size=file_size, duration=duration)
    files = server_control_client.get_or_create_files(epoch)
    epoch.urls = [server.file_to_url(file) for file in files]
    for benchmark_client in benchmark_clients:
        result = Result(
            server=server,
            client=benchmark_client,
            server_details=server_details,
            client_details=client_details,
            file_size=epoch.file_size,
            complete_size=epoch.complete_size,
        )
        result.elapsed = client_control_client.measure(benchmark_client, epoch)
        repository = SqliteRepository.build_repository("all_results.db")
        register_default_tables(repository)
        results = repository.tables["result"]
        result_id = results.add_result(result)
    server_control_client.stop_server(server)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results_from_database = results.get_results()

cols = set(["server", "client", "elapsed", "file_size_h", "bytes_per_second_h"])
pd.DataFrame(
    [
        {k: v for k, v in r.dict_for_pandas().items() if k in cols}
        for r in results_from_database
    ]
)

,server,client,elapsed,file_size_h,bytes_per_second_h
0,nginx/minimal,wrk,0.145824,97.66KB,2.39GB
1,nginx/minimal,httpx,4.373843,97.66KB,81.77MB
2,fastAPI/uvicorn,wrk,2.674689,97.66KB,133.71MB
3,fastAPI/uvicorn,httpx,5.065635,97.66KB,70.6MB
4,nginx/sendfile,wrk,0.116421,97.66KB,3.0GB
5,nginx/sendfile,httpx,4.177912,97.66KB,85.6MB
6,caddy,wrk,0.153288,97.66KB,2.28GB
7,caddy,httpx,4.324604,97.66KB,82.7MB


<IPython.core.display.Javascript object>